In [1]:
import requests
import pandas as pd
import numpy as np
import json

In [22]:
filepath = 'shopping_lists/shoppingList3.json'

In [23]:
with open(filepath) as f:
    shopping_list = json.load(f)

In [24]:
def get_items(shopping_list):
    items_df = pd.DataFrame([i[0] for i in pd.DataFrame(shopping_list['aisles'])['items']])
    qties = pd.DataFrame(list(items_df['measures']))['metric']
    items_df.drop(['measures', 'usages', 'pantryItem', 'cost'], axis=1, inplace=True)
    items_df['amount'] = [i['amount'] for i in qties]
    items_df['unit'] = [i['unit'] for i in qties]
    items_df['name'] = items_df['name'].map(str.split)
    return items_df

In [25]:
items_df = get_items(shopping_list)

In [26]:
items_df

,id,name,aisle,ingredientId,amount,unit
0,599451,"[bacon, fat]",Meat,4609,14.0,g
1,599483,"[tapioca, starch]",Ethnic Foods,93696,14.2,g
2,599491,"[fresh, thyme]",Produce,2049,1.1,g
3,599567,"[vanilla, extract]",Baking,2050,2.7,ml
4,599575,"[white, vinegar]","Oil, Vinegar, Salad Dressing",2053,3.8,ml
5,599581,"[english, muffin]",Bakery/Bread,18439,236.6,g
6,599583,"[canned, diced, tomatoes]",Canned and Jarred,11531,1.0,
7,599593,"[ground, cumin]",Spices and Seasonings,1002014,2.0,g
8,599621,[almond],Nuts,12061,143.0,g
9,599631,"[tomato, sauce]",Pasta and Rice,11549,99.3,g


In [8]:
r = requests.get('http://localhost:8000/products?format=json')

In [9]:
shop_df = pd.DataFrame(json.loads(r.content.decode()))

In [10]:
raw = shop_df.copy()

In [11]:
shop_df['name'] = [i.lower() for i in shop_df['name']]

In [12]:
shop_df.head()

,store,name,img_url,price,currency,amount,unit
0,sainsburys,sainsbury's fairtrade bananas loose,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,73.0,p,Loose,kg
1,sainsburys,sainsbury's carrots loose,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,41.0,p,Loose,kg
2,sainsburys,sainsbury's whole cucumber,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,43.0,p,,unit
3,sainsburys,sainsbury's onions loose,https://www.sainsburys.co.uk/wcsstore7.59.1/Sa...,85.0,p,Loose,kg
4,sainsburys,sainsbury's baking potatoes loose,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,64.0,p,Loose,kg


In [13]:
shop_df[shop_df['name'].str.contains('(?=.*black)(?=.*beans)')]

,store,name,img_url,price,currency,amount,unit
745,sainsburys,"sainsbury's black beans carton, so organic 380...",https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,75.0,p,(230g*),unit


In [27]:
matches_df = pd.DataFrame()
for product in items_df.iloc:
    regex = ''.join([f'(?=.*{i})' for i in product['name']])   
    entries = shop_df['name'].str.contains(regex)    
    if entries.any():
        matches = shop_df[entries]
        print(f"appending {' '.join(product['name'])}")
        matches_df = matches_df.append(matches)
    else:
        print(f"no such product: {' '.join(product['name'])}")

no such product: bacon fat
no such product: tapioca starch
no such product: fresh thyme
no such product: vanilla extract
no such product: white vinegar
appending english muffin
no such product: canned diced tomatoes
no such product: ground cumin
appending almond
appending tomato sauce
no such product: ground pepper
no such product: almond flour
no such product: dijon mustard
no such product: nondairy milk
appending honey
appending eggs
no such product: coconut aminos
appending dates
appending maple
no such product: pumpkin seeds


### add to the database:
    - ground cumin / cumin
    - hot sauce
    - almonds, agave
    - chili sauce
    - oats
    - peanut butter
    - tortilas
    - vanilla extract
    - starch
    - vinegar (white, apple cider)

### possibly drop stuff like 
    - quick 
    - canned
    - cooking 
    - fat

In [287]:
matches_df.sample(frac=0.2)

,store,name,img_url,price,currency,amount,unit
847,sainsburys,kitkat 4 finger milk chocolate bar multipack 4...,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,1.00,GBP,4x41.5g,unit
720,sainsburys,cadbury dairy milk chocolate bar 200g,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,1.50,GBP,200g,unit
723,sainsburys,cadbury dairy milk giant chocolate buttons bag...,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,1.50,GBP,119g,unit
901,sainsburys,cadbury dairy milk chocolate bar 10x29.3g,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,2.00,GBP,10x29.3g,unit
1089,sainsburys,"sainsbury's indulgence ice cream, milk chocola...",https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,1.35,GBP,4x110ml,unit
702,sainsburys,mcvitie's milk chocolate digestives biscuits 266g,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,1.60,GBP,266g,unit
632,sainsburys,sainsbury's toastie thick sliced white bread 800g,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,55.00,p,800g,unit
